In [ ]:
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

checkpoint = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to("cuda")

for param in model.base_model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt").to("cuda")
batch["labels"] = torch.tensor([1, 1]).to("cuda")

optimizer = AdamW(model.classifier.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3)

old_weights = {name: param.clone().detach() for name, param in model.named_parameters()}

model.train()
for _ in tqdm(range(100)):  # 100 эпох
    optimizer.zero_grad()

    loss = model(**batch).loss
    loss.backward()

    optimizer.step()
    scheduler.step(loss)

new_weights = {name: param.clone().detach() for name, param in model.named_parameters()}

old_weights['classifier.weight'], new_weights['classifier.weight']